## Start

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import glob
from scipy import optimize
import importlib

In [2]:
project_path = os.getcwd()

## load_data

### get_latest_position_file

In [56]:
data_dir = f'{project_path}/data'

In [57]:
files = glob.glob(os.path.join(data_dir, 'Portfolio_Positions_*.csv'))
files

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-06-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Aug-05-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Nov-16-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-29-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-02-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_May-06-2025.csv']

In [ ]:
latest_file = None
latest_date = None

for f in files:
    basename = os.path.basename(f)
    date_part = basename.replace('Portfolio_Positions_', '').replace('.csv', '')
    try:
        date_obj = datetime.strptime(date_part, '%b-%d-%Y')
        if latest_date is None or date_obj > latest_date:
            latest_date = date_obj
            latest_file = f
    except ValueError:
        continue
latest_date = pd.to_datetime(latest_date)
[latest_file, latest_date]

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 datetime.datetime(2025, 12, 31, 0, 0)]

### clean_positions

In [60]:
from support_functions.data_loader import get_latest_position_file, clean_currency

data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
positions_df = pd.read_csv(pos_file, index_col=False)
positions_df = positions_df.dropna(subset=['Account Name'])
cols_to_clean = [
    'Last Price', 'Current Value', 
    'Cost Basis Total', 'Today\'s Gain/Loss Dollar', 
    'Total Gain/Loss Dollar'
]

In [61]:
for col in cols_to_clean:
    if col in positions_df.columns:
        positions_df[col] = positions_df[col].apply(clean_currency)

# Clean Quantity (remove match for formatting issues if any)
if 'Quantity' in positions_df.columns:
    positions_df['Quantity'] = pd.to_numeric(positions_df['Quantity'], errors='coerce').fillna(0)

### load_transactions

In [41]:
data_dir = f'{project_path}/data'
hist_files = glob.glob(os.path.join(data_dir, 'Accounts_History_*.csv'))
transactions_dfs = []
print(f"Found {len(hist_files)} history files.")

for f in hist_files:
    df = pd.read_csv(f, on_bad_lines='skip') 
    transactions_dfs.append(df)

transactions_df = pd.concat(transactions_dfs, ignore_index=True)

Found 4 history files.


In [44]:
[transactions_df.iloc[292]['Run Date'],transactions_df.iloc[228]['Run Date']]

['11/13/2025', ' 02/15/2024']

### clean_transactions

In [ ]:
from support_functions import data_loader
importlib.reload(data_loader)
from support_functions.data_loader import load_transactions, clean_currency
data_dir = f'{project_path}/data'
transactions_df = load_transactions(data_dir)

Found 4 history files.


In [19]:
for col in transactions_df.columns:
    if (
        (transactions_df[col].dtype == 'object' ) or 
        (transactions_df[col].dtype == 'string')
    ):
        transactions_df[col] = transactions_df[col].str.strip()   
# Standardize dates
transactions_df['Run Date'] = pd.to_datetime(transactions_df['Run Date'], errors='coerce')
# Sometimes 'Settlement Date' exists
if 'Settlement Date' in transactions_df.columns:
        transactions_df['Settlement Date'] = pd.to_datetime(transactions_df['Settlement Date'], errors='coerce')
    


# Clean numeric columns
hist_numeric_cols = [
'Amount ($)', 'Price ($)', 'Quantity', 
    'Commission ($)', 'Fees ($)', 'Accrued Interest ($)'
]
for col in hist_numeric_cols:
    if col in transactions_df.columns:
        transactions_df[col] = transactions_df[col].apply(clean_currency)
    
# Sort by date
transactions_df = transactions_df.sort_values('Run Date')

In [21]:
transactions_df[transactions_df['Symbol'] == 'AAPL']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
537,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.0,164.61,0.0,0.0,0.0,-1481.45,2022-08-09
538,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.0,163.54,0.0,0.0,0.0,-163.54,2022-08-09
536,2022-08-26,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,164.50,0.0,0.0,0.0,-1645.00,2022-08-30
535,2022-08-29,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,160.00,0.0,0.0,0.0,-3200.00,2022-08-31
534,2022-11-01,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,150.00,0.0,0.0,0.0,-1500.00,2022-11-03
533,2022-11-08,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,138.50,0.0,0.0,0.0,-2770.00,2022-11-10
563,2022-11-10,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,11.50,NaT
629,2023-02-16,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.10,NaT
628,2023-05-18,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT
627,2023-08-17,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT


### load_data

In [3]:
from support_functions import analysis
importlib.reload(analysis)
from support_functions.analysis import (
    get_latest_position_file, clean_positions,
    load_transactions, clean_transactions
)

In [4]:
data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
print(f"Loading positions from: {pos_file} (Date: {pos_date.strftime('%Y-%m-%d')})")
positions_df = pd.read_csv(pos_file, index_col=False)

positions_df = clean_positions(positions_df)


Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)


In [5]:
transactions_df = load_transactions(data_dir)
    
transactions_df = clean_transactions(transactions_df)

Found 4 history files.


In [6]:
transactions_df[transactions_df['Symbol'] == 'AAPL']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
537,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.0,164.61,0.0,0.0,0.0,-1481.45,2022-08-09
538,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.0,163.54,0.0,0.0,0.0,-163.54,2022-08-09
536,2022-08-26,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,164.50,0.0,0.0,0.0,-1645.00,2022-08-30
535,2022-08-29,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,160.00,0.0,0.0,0.0,-3200.00,2022-08-31
534,2022-11-01,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,150.00,0.0,0.0,0.0,-1500.00,2022-11-03
533,2022-11-08,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,138.50,0.0,0.0,0.0,-2770.00,2022-11-10
563,2022-11-10,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,11.50,NaT
629,2023-02-16,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.10,NaT
628,2023-05-18,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT
627,2023-08-17,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT


## analyze_entity_performance

### filter_transactions

In [11]:
from support_functions.data_loader import load_data

In [ ]:
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [57]:
transactions_df[transactions_df['Account Number'].isna()]

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Asset Type
237,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
238,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
239,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
240,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
241,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
242,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
243,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
244,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
245,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock
520,NaT,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaT,Stock


In [ ]:
symbol = None
account_num = '86964'
df = transactions_df.copy()
df = df[
    (df['Account Number'] == account_num) &
    (df['Symbol'] == symbol)
]
df.head(4)




,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date,Asset Type


In [ ]:
account_num = 'Z23390746'
symbol = None
df = transactions_df.copy()
funding_patterns = [
    'ELECTRONIC FUNDS TRANSFER', 'CHECK RECEIVED', 'DEPOSIT', 'WIRE', 
    'BILL PAY', 'CONTRIB', 'PARTIC CONTR'
]
mask_account = (df['Account Number'] == account_num)
mask_pattern = df['Action'].str.upper().apply(lambda x: any(p in str(x) for p in funding_patterns))

df = df[mask_account & mask_pattern]
df['Amount ($)'].sum()

np.float64(1100000.0)

In [20]:
funding_patterns = ['ELECTRONIC FUNDS TRANSFER RECEIVED (CASH)']
df['Action'].str.upper().isin(funding_patterns)

553     True
562    False
545    False
531    False
530    False
       ...  
255    False
252    False
247    False
246    False
250    False
Name: Action, Length: 445, dtype: bool

### filter_positions

In [ ]:
from support_functions.data_loader import load_data
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [27]:
symbol = 'AAPL'
df = positions_df.copy()
df = df[df['Symbol'] == symbol]
df.head(4)

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
10,Z23390746,Individual,AAPL,APPLE INC,70.0,271.86,-$1.22,19030.2,-85.4,-0.45%,8270.21,+76.86%,1.52%,10759.99,$153.71,Cash


In [28]:
account_num = 'Z23390746'
df = positions_df.copy()
df = df[df['Account Number'] == account_num]
df.head(4)

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type
0,Z23390746,Individual,912797SG3,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,200000.000,99.823,+$0.018,199646.00,36.00,+0.01%,523.33,+0.26%,15.96%,199122.67,--,Cash
1,Z23390746,Individual,FXAIX,FIDELITY 500 INDEX FUND,749.338,237.720,-$1.74,178132.62,-1303.85,-0.73%,35568.68,+24.94%,14.24%,142563.94,$190.25,Cash
2,Z23390746,Individual,FZFXX**,HELD IN MONEY MARKET,0.000,0.000,NaN,128390.20,0.00,NaN,0.00,NaN,10.27%,0.00,NaN,Cash
3,Z23390746,Individual,FSKAX,FIDELITY TOTAL MARKET INDEX FUND,658.109,186.840,-$1.42,122961.08,-934.52,-0.76%,20661.19,+20.19%,9.83%,102299.89,$155.45,Cash


### build_stock_cash_flows

In [ ]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [4]:
symbol = 'AAPL'
account_num = 'Z23390746'

In [ ]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)

In [6]:
cash_flows = []
total_invested = 0.0
current_val = filtered_posi['Current Value'].iloc[0]
for _, row in filtered_hist.iterrows():
    date = row['Run Date']
    amount = row['Amount ($)']
    flow = amount
    cash_flows.append((date, flow))
    
    # Track Invested Capital (Sum of negative flows)
    if flow < 0:
        total_invested += abs(flow)

cash_flows.append((latest_date, current_val))


In [7]:
cash_flows

[(Timestamp('2022-08-05 00:00:00'), -1481.45),
 (Timestamp('2022-08-05 00:00:00'), -163.54),
 (Timestamp('2022-08-26 00:00:00'), -1645.0),
 (Timestamp('2022-08-29 00:00:00'), -3200.0),
 (Timestamp('2022-11-01 00:00:00'), -1500.0),
 (Timestamp('2022-11-08 00:00:00'), -2770.0),
 (Timestamp('2022-11-10 00:00:00'), 11.5),
 (Timestamp('2023-02-16 00:00:00'), 16.1),
 (Timestamp('2023-05-18 00:00:00'), 16.8),
 (Timestamp('2023-08-17 00:00:00'), 16.8),
 (Timestamp('2023-11-16 00:00:00'), 16.8),
 (Timestamp('2024-02-15 00:00:00'), 16.8),
 (Timestamp('2024-05-16 00:00:00'), 17.5),
 (Timestamp('2024-08-15 00:00:00'), 17.5),
 (Timestamp('2024-11-14 00:00:00'), 17.5),
 (Timestamp('2025-02-13 00:00:00'), 17.5),
 (Timestamp('2025-05-15 00:00:00'), 18.2),
 (Timestamp('2025-08-14 00:00:00'), 18.2),
 (Timestamp('2025-11-13 00:00:00'), 18.2),
 (Timestamp('2025-12-31 00:00:00'), np.float64(19030.2))]

### build_account_cash_flows

In [43]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [50]:
symbol = None
account_num = 'ERNST & YOUNG 401(K)'

In [51]:
filtered_hist = filter_transactions(transactions_df, account_num, symbol)
filtered_posi = filter_positions(positions_df, account_num, symbol)

In [49]:
positions_df

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis Total,Average Cost Basis,Type,Asset Type
0,Z23390746,Individual,912797SG3,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,200000.000,99.823,+$0.018,199646.00,36.00,+0.01%,523.33,+0.26%,15.96%,199122.67,--,Cash,Bond
1,Z23390746,Individual,FXAIX,FIDELITY 500 INDEX FUND,749.338,237.720,-$1.74,178132.62,-1303.85,-0.73%,35568.68,+24.94%,14.24%,142563.94,$190.25,Cash,Stock
2,Z23390746,Individual,FZFXX**,HELD IN MONEY MARKET,0.000,0.000,NaN,128390.20,0.00,NaN,0.00,NaN,10.27%,0.00,NaN,Cash,Cash
3,Z23390746,Individual,FSKAX,FIDELITY TOTAL MARKET INDEX FUND,658.109,186.840,-$1.42,122961.08,-934.52,-0.76%,20661.19,+20.19%,9.83%,102299.89,$155.45,Cash,Stock
4,Z23390746,Individual,912797SE8,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,100000.000,99.962,+$0.019,99962.00,19.00,+0.01%,544.85,+0.54%,7.99%,99417.15,--,Cash,Bond
5,Z23390746,Individual,912797RJ8,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 01...,100000.000,99.872,+$0.018,99872.00,18.00,+0.01%,303.67,+0.30%,7.99%,99568.33,--,Cash,Bond
6,Z23390746,Individual,912797SQ1,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 02...,100000.000,99.617,+$0.016,99617.00,16.00,+0.01%,178.56,+0.17%,7.96%,99438.44,--,Cash,Bond
7,Z23390746,Individual,912797SS7,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 02...,100000.000,99.480,+$0.016,99480.00,16.00,+0.01%,37.67,+0.03%,7.95%,99442.33,--,Cash,Bond
8,Z23390746,Individual,912797ST5,UNITED STATES TREAS BILLS ZERO CPN 0.00000% 03...,100000.000,99.408,+$0.013,99408.00,13.00,+0.01%,-35.11,-0.04%,7.95%,99443.11,--,Cash,Bond
9,Z23390746,Individual,FSPSX,FIDELITY INTERNATL INDEX FUND,1265.781,60.800,-$0.18,76959.48,-227.85,-0.30%,12965.08,+20.25%,6.15%,63994.40,$50.56,Cash,Stock


In [46]:
filtered_hist['Amount ($)'].sum()

np.float64(0.0)

In [7]:
cash_flows = []
total_invested = 0.0
current_val = filtered_posi['Current Value'].sum()
for _, row in filtered_hist.iterrows():
    date = row['Run Date']
    amount = row['Amount ($)']
    flow = -amount
    cash_flows.append((date, flow))
    
    # Track Invested Capital (Sum of negative flows)
    total_invested -= (flow)

cash_flows.append((latest_date, current_val))

In [8]:
total_invested

1100000.0

### xirr

In [15]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [18]:
symbol = None
account_num = 'Z23390746'

In [19]:
cash_flows, total_invested = build_account_cash_flows(transactions_df, positions_df, latest_date, account_num)

In [25]:
dates, amounts = width = zip(*cash_flows)
min_date = min(dates)
days = [(d - min_date).days for d in dates]

# Optimization function
def npv(r):
    arr = np.array(days)
    vals = np.array(amounts)
    return np.sum(vals / (1 + r)**(arr / 365.0))
    
# Check signs
pos = any(a > 0 for a in amounts)
neg = any(a < 0 for a in amounts)
res = optimize.newton(npv, 0.1, maxiter=50)
res

np.float64(0.09884212125959153)

### analyze_entity_performance

In [39]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

symbol = 'AAPL'
account_num = 'ERNST & YOUNG 401(K)'

cash_flows, total_invested = build_account_cash_flows(transactions_df, positions_df, latest_date, account_num)
irr = xirr(cash_flows)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [42]:
total_invested

0.0

In [ ]:
{
    'Account Name': account_name,
    'Account Number': account_num,
    'Symbol': symbol,
    'Asset Type': asset_type,
    'Current Value': current_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return_dollar,
    'Total Return (%)': total_return_pct,
    'IRR': irr_val if irr_val is not None else np.nan
}

10759.99

In [ ]:
from support_functions.analysis import (
    load_data,categorize_asset,xirr
)
data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)
results = []
for i, row in positions_df.iterrows():
    symbol = row['Symbol']
    account_num = row['Account Number']
    account_name = row['Account Name']
    current_val = row['Current Value']
    quantity = row['Quantity']
    asset_type = categorize_asset(row)
    
    # Filter History
    mask = (transactions_df['Account Number'] == account_num) & (transactions_df['Symbol'] == symbol)
    symbol_hist = transactions_df[mask]
    
    # Build Cash Flows
    cash_flows = []
    
    # 1. Transactions (Buys are negative amount, Sells/Divs are positive amount in Fidelity History)
    # Verify assumption:
    # Buy: Amount is negative (outflow).
    # Sell: Amount is positive (inflow).
    # Div: Amount is positive (inflow).
    # So we can sum 'Amount ($)' directly.
    
    relevant_actions = symbol_hist[symbol_hist['Amount ($)'].notna()].reset_index(drop=True)
    
    total_invested = 0
    total_returned = 0
    
    for _, h_row in relevant_actions.iterrows():
        date = h_row['Run Date']
        amt = h_row['Amount ($)']
        cash_flows.append((date, amt))
        
        if amt < 0:
            total_invested += abs(amt)
        else:
            total_returned += amt
            
    # 2. Add Current Value as a final "inflow" on the latest position date
    # Only if we currently hold it (Current Value > 0)
    # today = datetime.now() -> Changed to latest_date from file
    if current_val > 0:
        cash_flows.append((latest_date, current_val))
        
    # Metrics
    irr_val = xirr(cash_flows)
    
    # Total Return ($) = (Total Returned + Current Value) - Total Invested
    # Or simply Sum of all Cash Flows (since Flows include negative buys and positive sells/divs) + Current Value
    total_return_dollar = sum([cf[1] for cf in cash_flows]) # Note: cash_flows includes Current Value now
    
    # Return % = Total Return $ / Total Invested 
    # (Simple ROI, distinct from IRR)
    total_return_pct = (total_return_dollar / total_invested) if total_invested > 0 else 0
    
    results.append({
        'Account Name': account_name,
        'Account Number': account_num,
        'Symbol': symbol,
        'Asset Type': asset_type,
        'Current Value': current_val,
        'Total Invested': total_invested,
        'Total Return ($)': total_return_dollar,
        'Total Return (%)': total_return_pct,
        'IRR': irr_val if irr_val is not None else np.nan
    })
pd.DataFrame(results)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,Individual,Z23390746,912797SG3,Bond,199646.00,199122.67,523.33,0.002628,0.036137
1,Individual,Z23390746,FXAIX,Stock,178132.62,142563.94,38132.62,0.267477,0.222154
2,Individual,Z23390746,FZFXX**,Cash,128390.20,0.00,128390.20,0.000000,NaN
3,Individual,Z23390746,FSKAX,Stock,122961.08,102299.89,22961.08,0.224449,0.167138
4,Individual,Z23390746,912797SE8,Bond,99962.00,99417.15,544.85,0.005480,0.036937
5,Individual,Z23390746,912797RJ8,Bond,99872.00,99568.33,303.67,0.003050,0.039072
6,Individual,Z23390746,912797SQ1,Bond,99617.00,99438.44,178.56,0.001796,0.033284
7,Individual,Z23390746,912797SS7,Bond,99480.00,99442.33,37.67,0.000379,0.019945
8,Individual,Z23390746,912797ST5,Bond,99408.00,99443.11,-35.11,-0.000353,NaN
9,Individual,Z23390746,FSPSX,Stock,76959.48,63994.40,16959.48,0.265015,0.177839


In [43]:
total_return_dollar = sum([cf[1] for cf in cash_flows])
total_return_dollar

np.float64(8489.61)

In [45]:
total_return_pct = (total_return_dollar / total_invested) if total_invested > 0 else 0
total_return_pct

np.float64(0.7889979451653766)

In [46]:
{
    'Account Name': account_name,
    'Account Number': account_num,
    'Symbol': symbol,
    'Asset Type': asset_type,
    'Current Value': current_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return_dollar,
    'Total Return (%)': total_return_pct,
    'IRR': irr_val if irr_val is not None else np.nan
}

{'Account Name': 'Individual',
 'Account Number': 'Z23390746',
 'Symbol': 'AAPL',
 'Asset Type': 'Stock',
 'Current Value': np.float64(19030.2),
 'Total Invested': 10759.99,
 'Total Return ($)': np.float64(8489.61),
 'Total Return (%)': np.float64(0.7889979451653766),
 'IRR': np.float64(0.19542017278306092)}

## analyze_account_performance

In [6]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr, analyze_entity_performance
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [ ]:
accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()
accounts_unique

,Account Number,Account Name,Current Value
0,241802439,Health Savings Account,9707.44
1,86964,ERNST & YOUNG 401(K),29217.10
2,Z06872898,Cash Management (Individual),108.17
3,Z23390746,Individual,1250701.91


In [34]:
row = accounts_unique.iloc[1]
acc_num = row['Account Number']
acc_name = row['Account Name']
curr_val = row['Current Value']

In [35]:
total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=None)

In [ ]:
total_return = curr_val-total_invested
total_return_ratio = total_return/total_invested
{
    'Account Name': acc_name,
    'Account Number': acc_num,
    'Symbol': None,
    'Asset Type': 'Account',
    'Current Value': curr_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': total_return_ratio,
    'IRR': irr
}

{'Account Name': 'ERNST & YOUNG 401(K)',
 'Account Number': '86964',
 'Symbol': None,
 'Asset Type': 'Account',
 'Current Value': np.float64(29217.1),
 'Total Invested': 0.0,
 'Total Return ($)': np.float64(29217.1),
 'Total Return (%)': np.float64(inf),
 'IRR': None}

In [58]:
results = []
    
accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

for _, row in accounts_unique.iterrows():
    acc_num = row['Account Number']
    acc_name = row['Account Name']
    curr_val = row['Current Value']
    if acc_name == 'ERNST & YOUNG 401(K)':
        continue
    
    total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=None)
    total_return = curr_val-total_invested
    total_return_ratio = total_return/total_invested
    
    results.append({
        'Account Name': acc_name,
        'Account Number': acc_num,
        'Symbol': None,
        'Asset Type': 'Account',
        'Current Value': curr_val,
        'Total Invested': total_invested,
        'Total Return ($)': total_return,
        'Total Return (%)': total_return_ratio,
        'IRR': irr
    })
pd.DataFrame(results)

,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,Health Savings Account,241802439,None,Account,9707.44,8450.64,1256.80,0.148722,0.149874
1,Cash Management (Individual),Z06872898,None,Account,108.17,8.17,100.00,12.239902,2.463594
2,Individual,Z23390746,None,Account,1250701.91,1100000.00,150701.91,0.137002,0.098842


## analyze_stock_performance

In [59]:
from support_functions.data_loader import load_data
from support_functions.analysis import (
    filter_transactions, filter_positions, 
    build_stock_cash_flows, build_account_cash_flows,
    xirr, analyze_entity_performance
)

data_dir = f'{project_path}/data'
positions_df, transactions_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [61]:
results = []

accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

In [63]:
row = accounts_unique.iloc[3]
acc_num = row['Account Number']
acc_name = row['Account Name']

In [72]:
sub_positions_df = positions_df[
    (positions_df['Account Name'] == acc_name) &
    (positions_df['Asset Type'] == 'Stock')
]
sub_row = sub_positions_df.iloc[4]
symbol = sub_row['Symbol']
curr_val = sub_row['Current Value']


total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=symbol)
{
    'Account Name': acc_name,
    'Account Number': acc_num,
    'Symbol': symbol,
    'Asset Type': 'Stock',
    'Current Value': curr_val,
    'Total Invested': total_invested,
    'Total Return ($)': total_return,
    'Total Return (%)': total_return_ratio,
    'IRR': irr
}

{'Account Name': 'Individual',
 'Account Number': 'Z23390746',
 'Symbol': 'MSFT',
 'Asset Type': 'Stock',
 'Current Value': np.float64(14508.6),
 'Total Invested': 13342.95,
 'Total Return ($)': 150701.90999999992,
 'Total Return (%)': 0.1370017363636363,
 'IRR': np.float64(0.06671563300410761)}

In [74]:
results = []

accounts_unique = positions_df.groupby(['Account Number','Account Name'])['Current Value'].sum().reset_index()

for _, row in accounts_unique.iterrows():

    acc_num = row['Account Number']
    acc_name = row['Account Name']


    target_type = 'Stock'
    sub_positions_df = positions_df[
        (positions_df['Account Name'] == acc_name) &
        (positions_df['Asset Type'] == target_type)
    ]

    for _, sub_row in sub_positions_df.iterrows():
        symbol = sub_row['Symbol']
        curr_val = sub_row['Current Value']
        if symbol in ['Pending activity']:
            continue

        total_invested, irr = analyze_entity_performance(transactions_df, positions_df, latest_date, account_num=acc_num, symbol=symbol)
        results.append({
            'Account Name': acc_name,
            'Account Number': acc_num,
            'Symbol': symbol,
            'Asset Type': 'Stock',
            'Current Value': curr_val,
            'Total Invested': total_invested,
            'Total Return ($)': total_return,
            'Total Return (%)': total_return_ratio,
            'IRR': irr
        })
pd.DataFrame(results)

,Account Name,Account Number,Symbol,Asset Type,Current Value,Total Invested,Total Return ($),Total Return (%),IRR
0,Health Savings Account,241802439,FXAIX,Stock,4232.36,3560.87,150701.91,0.137002,0.167011
1,Health Savings Account,241802439,FZILX,Stock,4144.74,3836.14,150701.91,0.137002,0.300799
2,ERNST & YOUNG 401(K),86964,84679P173,Stock,20130.87,0.00,150701.91,0.137002,NaN
3,ERNST & YOUNG 401(K),86964,FBCGX,Stock,9086.23,0.00,150701.91,0.137002,NaN
4,Cash Management (Individual),Z06872898,CORE**,Stock,108.17,0.00,150701.91,0.137002,NaN
5,Individual,Z23390746,FXAIX,Stock,178132.62,142563.94,150701.91,0.137002,0.222154
6,Individual,Z23390746,FSKAX,Stock,122961.08,102299.89,150701.91,0.137002,0.167138
7,Individual,Z23390746,FSPSX,Stock,76959.48,63994.40,150701.91,0.137002,0.177839
8,Individual,Z23390746,AAPL,Stock,19030.20,10759.99,150701.91,0.137002,0.195420
9,Individual,Z23390746,MSFT,Stock,14508.60,13342.95,150701.91,0.137002,0.066716
